In [28]:
import pandas as pd
import os

# Definir la subcarpeta
#subfolder = '../../data_csv_bicing/'
subfolder = 'data/'

# Obtener la lista de archivos CSV en la subcarpeta
files = os.listdir(subfolder)
files = [f for f in files if f.endswith('ESTACIONS.csv')]


# Lista para almacenar los DataFrames de cada archivo CSV
df_list = []

# Leer cada archivo CSV y añadirlo a la lista
for file in files:
    file_path = os.path.join(subfolder, file)
    df_temp = pd.read_csv(file_path)
    df_list.append(df_temp)

# Concatenar todos los DataFrames en uno solo
big_df = pd.concat(df_list, ignore_index=True)

# Guardar en formato parquet
big_df.to_parquet('data/1_all_data_raw.parquet', index=False)


/var/folders/sv/pmf8l0nn4rs2zw5xdj5370vr0000gn/T/ipykernel_74832/3070975479.py:18: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(file_path)
/var/folders/sv/pmf8l0nn4rs2zw5xdj5370vr0000gn/T/ipykernel_74832/3070975479.py:18: DtypeWarning: Columns (7) have mixed types. Specify dtype option on import or set low_memory=False.
  df_temp = pd.read_csv(file_path)


In [1]:
import pandas as pd
import numpy as np

# Leer el archivo .parquet 
df = pd.read_parquet('data/1_all_data_raw.parquet')

In [2]:
# limpiar los datos
 
# Eliminar filas donde 'station_id' es nulo y pasarlos a int
df = df.dropna(subset=['station_id'])
df['station_id'] = df['station_id'].astype(int)

df['last_reported'] = pd.to_datetime(df['last_reported'], unit='s')

# Timestamp to hour, day, month, year
df['hour'] = df['last_reported'].dt.hour
df['day'] = df['last_reported'].dt.day
df['month'] = df['last_reported'].dt.month
df['year'] = df['last_reported'].dt.year

# seleccionar columnas relevantes
cols_to_keep = ['station_id', 'year', 'month', 'day', 'hour', 'num_bikes_available', 'num_bikes_available_types.mechanical', 'num_bikes_available_types.ebike','num_docks_available', 'last_reported']
df = df[cols_to_keep]

# Merge by taking the mean of the values
df = df.groupby(['station_id', 'hour', 'day', 'month', 'year']).mean().reset_index()

# Guardar en formato parquet
df.to_parquet("data/2_all_data_mean_hour.parquet", index=False)

In [48]:
import pandas as pd
import numpy as np

# Leer el archivo .parquet 
df_clean = pd.read_parquet('data/2_all_data_mean_hour.parquet')

# Cargar datos de las estaciones
df_station_info = pd.read_csv('data/Informacio_Estacions_Bicing.csv')

In [47]:
# Verificar si hay alguna columna relevaante con valores nulos
hay_nulos1 = df_clean.isnull().any()
hay_nulos2 = df_station_info.isnull().any()

print(hay_nulos1, hay_nulos2)

station_id                              False
hour                                    False
day                                     False
month                                   False
year                                    False
num_bikes_available                     False
num_bikes_available_types.mechanical    False
num_bikes_available_types.ebike         False
num_docks_available                     False
last_reported                           False
dtype: bool station_id                False
name                      False
physical_configuration    False
lat                       False
lon                       False
altitude                  False
address                   False
post_code                 False
capacity                  False
is_charging_station       False
nearby_distance           False
_ride_code_support        False
rental_uris                True
cross_street               True
dtype: bool


In [57]:

# se tienen que eliminar filas con numeros de station_id inexistentes en el registro de estaciones.
df_merge = df_clean.merge(df_station_info[["station_id", "capacity"]],
                             on="station_id",
                             how="inner")

df_merge["percentage_docks_available"] = df_merge["num_docks_available"] / df_merge["capacity"]
df_merge.to_parquet("data/3_all_data_ctx.parquet", index=False)


In [58]:
import pandas as pd

# Leer el archivo .parquet 
df_merge = pd.read_parquet('data/3_all_data_ctx.parquet')

In [68]:
# TODO revisar estrategia.

prediction_data = pd.DataFrame()

for s in df_merge.station_id.unique():
    ctx = df_merge.loc[df_merge["station_id"] == s, :]
    ctx = ctx.sort_values(by=["year", "month", "day", "hour"],
                            ignore_index=True)

    for lag in range(1, 5):
        ctx.loc[:, f"ctx-{lag}"] = ctx.loc[:, "percentage_docks_available"].shift(lag)

    ctx = ctx.iloc[4::5]

    prediction_data = pd.concat([prediction_data, ctx], ignore_index=True)

prediction_data = prediction_data[["station_id", "year", "month", "day", "hour", "percentage_docks_available"]]

prediction_data.to_parquet("data/4_data_ctx4h_v1.parquet", index=False)